#Instalar modulos

In [36]:
!pip install pandas==1.3.5
!pip install openpyxl==3.0.9
!pip install xlrd==2.0.1
!pip install pyreadstat

#Importar módulos

In [37]:
import pandas as pd
import numpy as np
import os
from os.path import exists
import inspect
import datetime
import csv
import pyreadstat

#Conexión a Google Drive

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Variables

In [39]:
path = "/content/drive/Shared drives/BID/Ciudades/"
path_data = path + "MicroDatos/"
path_salida = path + "Salida/Extraccion_v2/"
file = path + "parametros.xlsx"
encoding='iso-8859-1'

# Dataframes

In [41]:
df_paises = pd.read_excel(file, sheet_name = 'paises')
#df_obj = df_paises.select_dtypes(['object'])
#df_paises[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
df_paises['pais'] = df_paises['pais'].str.strip()
df_paises['carpeta'] = df_paises['carpeta'].str.strip()
df_paises = df_paises[df_paises['procesar'] == 1]

df_ciudades = pd.read_excel(file, sheet_name = 'ciudades')
df_ciudades['pais'] = df_ciudades['pais'].str.strip()

df_sectores = pd.read_excel(file, sheet_name = 'sectores_ok')
df_sectores['pais'] = df_sectores['pais'].str.strip()
if df_sectores['cod_ciiu'].dtypes == object:
    df_sectores["cod_ciiu"] = pd.to_numeric(df_sectores["cod_ciiu"], downcast="integer", errors='coerce')
    df_sectores["cod_ciiu"] = df_sectores["cod_ciiu"].fillna(0).astype(int)
df_sectores = df_sectores[['pais', 'cod_ciiu', 'sector']]

df_archivos = pd.read_excel(file, sheet_name = 'archivos')
#df_archivos = pd.read_excel(file, sheet_name = 'archivos_pruebas')
#df_obj = df_archivos.select_dtypes(['object'])
#df_archivos[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
df_archivos = df_archivos.sort_values(['carpeta', 'year', 'archivo'])

try:
    df_archivos["ciudad"] = df_archivos["ciudad"].str.lower()
    df_archivos["sector"] = df_archivos["sector"].str.lower()
    df_archivos["ocupado"] = df_archivos["ocupado"].str.lower()
    df_archivos["salario"] = df_archivos["salario"].str.lower()
    df_archivos["factor_exp"] = df_archivos["factor_exp"].str.lower()
except:
    pass

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Verificar dataframes

In [42]:
df_paises
#df_sectores
#df_sectores.sector.unique()
#df_paises[df_paises['procesar']==1]
#df_archivos
#df_archivos[df_archivos['carpeta'] == 'Argentina']
#df_ciudades
#df_ciudades[['codigo_ciudad','nombre']][df_ciudades['pais'] == 'El Salvador']
#df_ciudades.loc[df_ciudades['pais'] == 'Perú', ['codigo_ciudad','nombre']]

,pais,carpeta,procesar,ok,multiple,ocupado_valor,suelo_valor,mes_valor,Observaciones,Responble ajuste,Unnamed: 10,Observaciones Daniel
4,Colombia,Colombia,1.0,1.0,0.0,1.0,1.0,NaN,Sectores utiliza dos versiones de ciiu. Versio...,D,NaN,"No existen las columnas: inglabo (2020), revis..."


#**Nuevo Procedimiento**

In [43]:
debug=0 #1=imprime todos los mensajes
pd.options.mode.chained_assignment = None #'warn'
carpeta=""
pais=""
pais_ant=""
anio_ant=""
trimestre_ant=""
archivo=""
orden_salida=0
columns_rename={}
columns_order=['pais', 'anio', 'trimestre' ,'cod_ciudad', 'ciudad', 'cod_sector', 'sector', 'edad', 'sexo', 'ocupado', 'salario', 'factor_exp']
columns_conteo = 0 
columns_conteo_check = 7
columns_conteo_ini = 7
check_salario = 0

class bcolors:
    OK = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    BLUE = '\033[94m'
    ENDC = '\033[0m'

def time_dif(time1):
    time2 = datetime.datetime.now()
    td = time2 - time1
    td_mins = int(td.total_seconds()) // 60
    td_seconds = int(td.total_seconds()) % 60
    print("tiempo de ejecución:", td_mins, "m.", td_seconds, "s.", ' --- ', time2)

def print_var(p1, p2=None, p3=None, p4=None, p5=None):
    if debug==1:
        frame = inspect.currentframe()
        frame = inspect.getouterframes(frame)[1]
        string = inspect.getframeinfo(frame[0]).code_context[0].strip()
        args = string[string.find('(') + 1:-1].split(',')
        names = []
        j = 1
        for i in args:
            p = eval("p"+str(j))
            var = i + ":" + str(p)
            names.append(var)
            j+=1
        names = " ".join(str(x) for x in names)
        print(names)

def print_s(msg):
    i = 100
    j = (i - len(msg)) // 2
    if (i - len(msg)) % 2 == 0:
        k = j
    else:
        k = j + 1
    print("")
    print("#"*j, msg, "#"*k)

def read_df(file):
    df = []
    filename, file_extension = os.path.splitext(file)
    file_extension = file_extension[1:]
    file_extension = file_extension.lower()
    print_var(file)
    try:
        if file_extension == "csv":
            try:
                reader = pd.read_csv(file, sep = None, iterator = True, engine='python')
                inferred_sep = reader._engine.data.dialect.delimiter
                df = pd.read_csv(file, sep=inferred_sep, low_memory=False, encoding=encoding)
            except:
                try:    
                    df = pd.read_csv(file, sep='[,;]', quotechar='"', quoting=csv.QUOTE_NONE, engine='python', encoding=encoding) 
                except:
                    df = pd.read_csv(file, sep='[,;]', index_col=False, quotechar='"', quoting=csv.QUOTE_NONE, engine='python', encoding=encoding)
                    df.drop(df.columns[0], axis=1, inplace=True)
        elif file_extension in ("txt"):
            if os.stat(file).st_size == 0:
                print("No existen registros")
            else:
                try:
                    df = pd.read_csv(file, sep=' ', low_memory=False)
                except:
                    df = pd.read_csv(file, sep=' ', index_col=False, low_memory=False, engine='python', encoding=encoding)
                    df.drop(df.columns[0], axis=1, inplace=True)
        elif file_extension in ("xls", "xlsx"):
            if col_hoja == col_hoja:
                df = pd.read_excel(file, sheet_name=col_hoja) 
            else:
                df = pd.read_excel(file, sheet_name=0)
        elif file_extension in ("sav"):
            #df = pd.read_spss(file)
            df,meta= pyreadstat.read_sav(file)
        elif file_extension in ("dta"):
            df = pd.read_stata(file)
        else:
            print("Archivo no reconocido")
    except Exception as e:
        print("Archivo leyendo archivo: " + str(e))
    return df

def columns_df(df):
    error_columns=0
    columns=list()
    columns_error=list()
    columns_rename.clear()
    
    df.columns = df.columns.str.strip('"')
    columns_all=df.columns
    columns_all_s=", ".join(str(x) for x in columns_all)
    #print_var(columns_all)
    #print_var(columns_all_s)
    
    #print_var(col_ciudad)
    #print_var(df.columns)
    if col_ciudad in df.columns:
        columns.append(col_ciudad)
        columns_rename[col_ciudad] = "cod_ciudad"
    elif col_ciudad != col_ciudad:
        pass
    else:
        columns_error.append(col_ciudad)
        
    if col_ocupado in df.columns:
        columns.append(col_ocupado)
        columns_rename[col_ocupado] = "ocupado"
    elif col_ocupado != col_ocupado:
        pass
    else:
        columns_error.append(col_ocupado)
        
    if col_sector in df.columns:
        columns.append(col_sector)
        columns_rename[col_sector] = "cod_sector"
    elif col_sector != col_sector:
        pass
    else:
        columns_error.append(col_sector)
        
    if col_edad in df.columns:
        columns.append(col_edad)
        columns_rename[col_edad] = "edad"
    elif col_edad != col_edad:
        pass
    else:
        columns_error.append(col_edad)
        
    if col_sexo in df.columns:
        columns.append(col_sexo)
        columns_rename[col_sexo] = "sexo"
    elif col_sexo != col_sexo:
        pass
    else:
        columns_error.append(col_sexo)
        
    if col_salario in df.columns:
        columns.append(col_salario)
        columns_rename[col_salario] = "salario"
        check_salario = 0
    elif col_salario != col_salario:
        check_salario = 1
        pass
    else:
        columns_error.append(col_salario)
        
    if col_factor_exp in df.columns:
        columns.append(col_factor_exp)
        columns_rename[col_factor_exp] = "factor_exp"
    elif col_factor_exp != col_factor_exp:
        pass
    else:
        columns_error.append(col_factor_exp)

    if col_llave == col_llave:
        columns.append(col_llave)
                        
    #print_var(columns)
    print_var(columns_rename)
    print_var(columns_error)
    columns_error = " ".join(str(x) for x in columns_error)
    return columns, columns_error, columns_all_s, check_salario

# Tmp

In [69]:
########## Mexico 2018
path = "/content/drive/Shared drives/BID/Ciudades/"
path_data = path + "MicroDatos/Mexico/"
file = path_data + 'coe1t218.csv'
df_data1 = read_df(file)
file = path_data + 'coe2t218.csv'
df_data2 = read_df(file)
file = path_data + 'sdemt218.csv'
df_data3 = read_df(file)

col_llave = 'CD_A, ENT, CON, V_SEL, N_HOG, H_MUD, N_REN'.lower()

df_data1 = df_data1[['p1', 'p4a', 'fac', 'cd_a', 'ent', 'con', 'v_sel', 'n_hog', 'h_mud', 'n_ren']]
df_data2 = df_data2[['p6c', 'ent', 'con', 'v_sel', 'n_hog', 'h_mud', 'n_ren']]
df_data3 = df_data3[['eda', 'sex', 'ent', 'con', 'v_sel', 'n_hog', 'h_mud', 'n_ren']]

df_data = pd.merge(df_data1, df_data2, on=['ent', 'con', 'v_sel', 'n_hog', 'h_mud', 'n_ren'])
df_data = pd.merge(df_data, df_data3, on=['ent', 'con', 'v_sel', 'n_hog', 'h_mud', 'n_ren'])

file_out=path_data+"2018.csv"
df_data.to_csv(file_out, encoding=encoding)


KeyError: ignored

# Validaciones

In [28]:
col_llave

'cd_a, ent, con, v_sel, n_hog, h_mud, n_ren'

In [70]:
df_data1

,R_DEF,CD_A,ENT,CON,UPM,D_SEM,N_PRO_VIV,V_SEL,N_HOG,H_MUD,...,P5F9,P5F10,P5F11,P5F12,P5F13,P5F14,P5F15,P5F99,UR,FAC
0,0,1,9,40001,900471,101,9,1,1,0,...,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,1,482
1,0,1,9,40001,900471,101,9,1,1,0,...,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,1,482
2,0,1,9,40001,900471,101,9,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,482
3,0,1,9,40001,900471,101,9,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,482
4,0,1,9,40001,900471,101,34,2,1,0,...,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,1,482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309610,0,86,32,60039,3260283,503,237,19,1,0,...,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,2,252
309611,0,86,32,60039,3260283,503,237,19,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,252
309612,0,86,32,60039,3260283,503,237,19,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,252
309613,0,86,32,60039,3260283,503,275,20,1,0,...,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,2,252


In [55]:
df_data1

,p1,p4a,cd_a,fac,cd_a,ent,con,v_sel,n_hog,h_mud,n_ren
0,1,8121.0,1,648,1,9,40001,2,1,0,1
1,2,NaN,1,648,1,9,40001,2,1,0,2
2,1,5411.0,1,648,1,9,40001,2,1,0,3
3,2,NaN,1,648,1,9,40001,3,1,0,1
4,1,8130.0,1,648,1,9,40001,3,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...
312028,2,NaN,86,239,86,32,60039,17,1,0,2
312029,1,1110.0,86,239,86,32,60039,18,1,0,1
312030,2,NaN,86,239,86,32,60039,18,1,0,2
312031,2,NaN,86,239,86,32,60039,20,1,2,1


In [54]:
df_data2

,p6c,cd_a,ent,con,v_sel,n_hog,h_mud,n_ren
0,,1,9,40001,2,1,0,1
1,,1,9,40001,2,1,0,2
2,8,1,9,40001,2,1,0,3
3,,1,9,40001,3,1,0,1
4,8,1,9,40001,3,1,0,2
...,...,...,...,...,...,...,...,...
312028,,86,32,60039,17,1,0,2
312029,,86,32,60039,18,1,0,1
312030,,86,32,60039,18,1,0,2
312031,,86,32,60039,20,1,2,1


In [51]:
print(len(df_data1.index))
print(len(df_data2.index))
print(len(df_data3.index))

312033
312033
398017


In [ ]:
df_data3.index.tolist()